In [1]:
import os
import pandas as pd
import sqlite3
import re
from pythainlp.util import normalize

In [2]:
db_path = os.path.join(".","db-lastest.db")

In [6]:
cv_path = os.path.join(".","cv-corpus-9.0-2022-04-27","th") # config your path commonvoice

In [7]:
validated_path = os.path.join(cv_path,"validated.tsv")

In [8]:
df = pd.read_csv(validated_path,sep="\t")

/opt/conda/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [9]:
df['path'][109682]

'common_voice_th_26565507.mp3'

In [12]:
!cp {os.path.join(cv_path,"clips","common_voice_th_26583620.mp3")} common_voice_th_26583620.mp3

In [13]:
df["sentence"][0]

'ใครเป็นผู้รับ'

In [14]:
conn = sqlite3.connect(db_path)

In [15]:
cursor = conn.execute("SELECT sent,sent_fix  from cv")

In [16]:
regex =re.compile("[^\w\u0E00-\u0E7Fa-zA-Z]")

In [17]:
regex.sub(" ","<ss></ss>แมว ]! กิน ปลา")

' ss   ss แมว    กิน ปลา'

In [18]:
def call(s):
    s=normalize(s.strip())
    return re.sub("\s\s+", " ",regex.sub(" ",s).replace("  "," ").replace("“","").replace(" ”","").replace("”","")).strip()

In [19]:
def call2(s):
    s=normalize(s.strip())
    return re.sub("\s\s+", " ",regex.sub(" ",s).replace("  "," ").replace("“","").replace(" ”","").replace("”","")).strip()

In [20]:
sent_fix= {}

In [21]:
for row in cursor:
    if row[1] != None:
        sent_fix[call(row[0])] = call(row[1])
    else:
        sent_fix[call(row[0])] = call(row[0])

In [22]:
conn.close()

In [23]:
def check(i):
    i = i.replace("ํา","ำ")
    t = call2(i)
    if t not in sent_fix.keys():
        return None
    m= sent_fix[t]
    if re.search(r'[a-zA-Z]',m):
        return None
    return m

In [60]:
replace_sent_df = pd.read_csv(os.path.join(".","list_file.tsv"),sep="\t")
file_replace={}
for index, row in replace_sent_df.iterrows():
    file_replace[row['file']]=row['sentence']

In [65]:
def replace_sent(filename):
    if filename in file_replace.keys():
        return file_replace[filename]
    else:
        return None

In [66]:
def check_all_ok(i,filename):
    c1 = check(i)
    if c1==None and filename in file_replace.keys():
        c1=replace_sent(filename)
    return c1

In [67]:
from tqdm.autonotebook import tqdm

/tmp/ipykernel_125682/987820437.py:1: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [68]:
df["sentence_fix"] = [check_all_ok(i,filename) for i,filename in tqdm(zip(list(df["sentence"]),list(df['path'])))]

0it [00:00, ?it/s]

In [59]:
# import IPython
# IPython.display.Audio(os.path.join(cv_path,"clips","common_voice_th_26177956.mp3"))

In [75]:
df[df['sentence_fix'].isna()]

,client_id,path,sentence,up_votes,down_votes,age,gender,accents,locale,segment,sentence_fix
15414,3af01f147320b41b9e99dbc32c4297b3f651fdd2de2991...,common_voice_th_25668677.mp3,Mr Lincoln เป็นที่ปรึกษาของฉัน,2,0,NaN,NaN,NaN,th,NaN,None
66795,c5641e0b40dd2daee49df98ba9af1c5bee1e886d00e252...,common_voice_th_24685112.mp3,Mr Lincoln เป็นที่ปรึกษาของฉัน,2,1,NaN,NaN,NaN,th,NaN,None
69839,035df65951a3c51a8efcbdaa7184f1fbe15d00ab5eaa3b...,common_voice_th_25635358.mp3,Mr Lincoln เป็นที่ปรึกษาของฉัน,2,1,thirties,male,NaN,th,NaN,None


In [76]:
# j=0
# k=0
# for index, row in df.iterrows():
#     i = row['sentence']
#     if call2(i) not in sent_fix.keys():
#         print(row['path'],row['sentence'])
#         j+=1
#     k+=1

In [70]:
j

31

In [71]:
df

,client_id,path,sentence,up_votes,down_votes,age,gender,accents,locale,segment,sentence_fix
0,00086f5dc46f9038f13bbd829c4118fab3ac28688d5ee8...,common_voice_th_25695281.mp3,ใครเป็นผู้รับ,2,0,NaN,NaN,NaN,th,NaN,ใครเป็นผู้รับ
1,00245363f02e6f86a15de6793dc282d6f7095a8db6889b...,common_voice_th_31277118.mp3,รู้ได้ไงว่าเขาไม่หนุก,2,0,NaN,NaN,NaN,th,NaN,รู้ได้ไงว่าเขาไม่หนุก
2,005237750a4d82bf743fe1e6eb0e4a579f0eb6a2455aaf...,common_voice_th_25722081.mp3,การที่จะทำเค้กแต่งงานชั้นเลิศคุณจะต้องใช้น้ำตา...,3,0,NaN,NaN,NaN,th,NaN,การที่จะทำเค้กแต่งงานชั้นเลิศคุณจะต้องใช้น้ำตา...
3,00d8e38f434aecc821a4b83ca441d1e780de1e71cae734...,common_voice_th_25695038.mp3,ผู้ชายคือช้างเท้าหน้า แต่ผู้หญิงคือควาญช้าง,2,1,NaN,NaN,NaN,th,NaN,ผู้ชายคือช้างเท้าหน้า แต่ผู้หญิงคือควาญช้าง
4,00f387052dfafcbf01aff9ec022d9bf7822203ae4e37f6...,common_voice_th_25784745.mp3,มันช่วยให้ฉันจำได้,3,0,NaN,NaN,NaN,th,NaN,มันช่วยให้ฉันจำได้
...,...,...,...,...,...,...,...,...,...,...,...
125222,765ddb5b81ef7881ddadd9c2d01f0a368270cf136bb407...,common_voice_th_31394529.mp3,ทริปนี้มีประมาณ ห้าสิบ คน ต่างวัยกันไป,2,0,fifties,male,NaN,th,NaN,ทริปนี้มีประมาณ ห้าสิบ คน ต่างวัยกันไป
125223,765ddb5b81ef7881ddadd9c2d01f0a368270cf136bb407...,common_voice_th_31394598.mp3,ทั้งส่วนกลาง,2,0,fifties,male,NaN,th,NaN,ทั้งส่วนกลาง
125224,765ddb5b81ef7881ddadd9c2d01f0a368270cf136bb407...,common_voice_th_31394628.mp3,ตำรวจประกาศยกเลิกด่านจราจรทั่วประเทศ,2,0,fifties,male,NaN,th,NaN,ตำรวจประกาศยกเลิกด่านจราจรทั่วประเทศ
125225,765ddb5b81ef7881ddadd9c2d01f0a368270cf136bb407...,common_voice_th_31394699.mp3,ที่นักวิจัยคนอื่นสามารถนำไปต่อยอดได้,2,0,fifties,male,NaN,th,NaN,ที่นักวิจัยคนอื่นสามารถนำไปต่อยอดได้


In [72]:
df1 = df[df['sentence_fix'].notna()]

In [73]:
df1

,client_id,path,sentence,up_votes,down_votes,age,gender,accents,locale,segment,sentence_fix
0,00086f5dc46f9038f13bbd829c4118fab3ac28688d5ee8...,common_voice_th_25695281.mp3,ใครเป็นผู้รับ,2,0,NaN,NaN,NaN,th,NaN,ใครเป็นผู้รับ
1,00245363f02e6f86a15de6793dc282d6f7095a8db6889b...,common_voice_th_31277118.mp3,รู้ได้ไงว่าเขาไม่หนุก,2,0,NaN,NaN,NaN,th,NaN,รู้ได้ไงว่าเขาไม่หนุก
2,005237750a4d82bf743fe1e6eb0e4a579f0eb6a2455aaf...,common_voice_th_25722081.mp3,การที่จะทำเค้กแต่งงานชั้นเลิศคุณจะต้องใช้น้ำตา...,3,0,NaN,NaN,NaN,th,NaN,การที่จะทำเค้กแต่งงานชั้นเลิศคุณจะต้องใช้น้ำตา...
3,00d8e38f434aecc821a4b83ca441d1e780de1e71cae734...,common_voice_th_25695038.mp3,ผู้ชายคือช้างเท้าหน้า แต่ผู้หญิงคือควาญช้าง,2,1,NaN,NaN,NaN,th,NaN,ผู้ชายคือช้างเท้าหน้า แต่ผู้หญิงคือควาญช้าง
4,00f387052dfafcbf01aff9ec022d9bf7822203ae4e37f6...,common_voice_th_25784745.mp3,มันช่วยให้ฉันจำได้,3,0,NaN,NaN,NaN,th,NaN,มันช่วยให้ฉันจำได้
...,...,...,...,...,...,...,...,...,...,...,...
125222,765ddb5b81ef7881ddadd9c2d01f0a368270cf136bb407...,common_voice_th_31394529.mp3,ทริปนี้มีประมาณ ห้าสิบ คน ต่างวัยกันไป,2,0,fifties,male,NaN,th,NaN,ทริปนี้มีประมาณ ห้าสิบ คน ต่างวัยกันไป
125223,765ddb5b81ef7881ddadd9c2d01f0a368270cf136bb407...,common_voice_th_31394598.mp3,ทั้งส่วนกลาง,2,0,fifties,male,NaN,th,NaN,ทั้งส่วนกลาง
125224,765ddb5b81ef7881ddadd9c2d01f0a368270cf136bb407...,common_voice_th_31394628.mp3,ตำรวจประกาศยกเลิกด่านจราจรทั่วประเทศ,2,0,fifties,male,NaN,th,NaN,ตำรวจประกาศยกเลิกด่านจราจรทั่วประเทศ
125225,765ddb5b81ef7881ddadd9c2d01f0a368270cf136bb407...,common_voice_th_31394699.mp3,ที่นักวิจัยคนอื่นสามารถนำไปต่อยอดได้,2,0,fifties,male,NaN,th,NaN,ที่นักวิจัยคนอื่นสามารถนำไปต่อยอดได้


In [74]:
df1.to_csv(os.path.join(cv_path,"validated_clean.tsv"), sep="\t", index=False)